# Pileup
*856114 - Costantini Davide*  
*869877 - Giannaccari Mattia*  
Progetto di laboratorio per il corso di bioinformatica: implementazione della pileup

----

Come prima cosa si installi pysam per la lettura del file BAM/SAM

In [ ]:
!conda install -y -c pysam

Si importino i moduli:
 * `pysam` per l'__indicizzazione__ del file
 * `AlignmentFile` per la __lettura__ del file
 * `re` per l'uso delle __espressioni regolari__
 * `counter` per contare il __numero di basi__ per ogni posizione

In [ ]:
import pysam
from pysam import AlignmentFile
import re
from collections import Counter

Si imposti il parametro sulla qualità minima degli allineamenti: una base della read che abbia qualità __inferiore alla soglia__ verrà ignorata.  
(Il valore 2 corrisponde al carattere # nella quality string della query, il calcolo verrà maggiormente approfondito nel seguito)

In [ ]:
quality_threshold = 2

Si imposti il parametro sul nome del file e si osservi inoltre che nella lettura del file può essere utilizzata la modalità di apertura `r` per file SAM, ovvero in __formato testuale__, oppure `rb` per file BAM, ovvero in __formato binario__. Tale modalità è impostata automaticamente in questa implementazione a partire dall'estensione del file aperto.

La principale differenza su cui è stata posta l'attenzione durante l'implementazione riguarda la __modalità di indirizzamento__ della reference e delle read nelle query:
* Un file in formato __SAM__ usa un sistema di coordinate __1-based__, ovvero un sistema in cui la prima base di una stringa corrisponde all'indice 1
* Un file in formato __BAM__ usa un sistema di coordinate __0-based__, ovvero un sistema in cui la prima base di una stringa corrisponde all'indice 0

In [ ]:
file_name = "sample.bam"
open_mode = "rb" if file_name.split('.')[-1] == "bam" else 'r'

Si indicizzi il file BAM tramite la funzione `index()` e si costruisca l'oggetto `AlignmentFile`.

In [ ]:
pysam.index(file_name)
bamfile = AlignmentFile(file_name, open_mode)

Si usi la funzione `fetch()` per ottenere un __iteratore__ contenente tutti gli allineamenti del file BAM.

In [ ]:
all_alignments = bamfile.fetch()

In questa impolementazione è stato utilizzato un __dizionario__ per memorizzare le basi delle read in funzione del loro allineamento nella reference: la __chiave__ del dizionario corrisponde alla __posizione nella reference__ e il __valore__ corrisponde a una __lista di tuple__ nel formato:  

`(QueryName, Base, Quality)`  

dove:
* __QueryName__ rappresenta il nome della query contenente la read da cui la base è stata letta
* __Base__ rappresenta la base impilata
* __Quality__ rappresenta la qualità della base, il cui calcolo verrà approfondito nel seguito

In [ ]:
pileup = dict()

Il valore della __qualità__ per ogni base della query si trova all'interno della __stringa di qualità__. Tale stringa ha una lunghezza pari alla lunghezza della read e il carattere i-esimo della stringa di qualità corrisponde al valore di qualità della base i-esima nella read.
I valori di qualità sono espressi come __caratteri ASCII__ compresi tra `!` e `~` e possono essere __convertiti in valori interi__ compresi tra 0 e 126 tramite l'espressione:

`ord(qual_string[offset]) - 33`

dove si legge il carattere all'indice `offset` della stringa `qual_string` e, tramite la funzione `ord()` si calcola il suo valore intero corrispondente nella tabella ASCII. A tale valore  si sottrae poi 33.

In [ ]:
def phred_to_int(quality):
    return ord(quality) - 33

Iterando su tutte query del file BAM usiamo la variabile `position` per indicizzare la __reference__ e la variabile `offset` per indicizzare la __read__.

L'espressione regolare `([0-9]+)([MIDNSHP])` è usata per __processare la CIGAR string__ ed ottenere una lista di tuple nel formato  

`(aligned_chars, alignment_type)`  

dove `aligned_chars` indica il __numero dei caratteri__ allineati e `alignment_type` il __tipo di allineamento__.  
Le possibili CIGAR operation sono:

| Operazione | Descrizione | Consuma la query | Consuma la reference |
|:----:|:---|:----:|:----:|
| __M__ | Match o mismatch | ✓ | ✓ |
| __I__ | Inserimento nella reference | ✓ |  |
| __D__ | Delezione dalla reference |  | ✓ |
| __N__ | Regione della reference saltata |  | ✓ |
| __S__ | Soft clipping (sequenza clippata presente nella read) | ✓ |  |
| __H__ | Hard clipping (sequenza clippata NON presente nella read) |  |  |
| __P__ | Padding (delezione silente dalla padded reference) |  |  |


Nel caso in cui l'operazione __consumi la reference__, viene incrementato l'indice `position`, nel caso in cui l'operazione __consumi la query__ viene incrementato l'indice `offset`.

Si osservi inoltre che, data una certa query, la somma delle operazioni  `M`, `I`, `S` deve essere uguale alla lunghezza della read associata a tale query.

La base viene dunque impilata solo nel caso in cui l'operazione CIGAR sia `M` e tale base abbia una qualità __maggiore o uguale__ alla soglia di qualità `quality_threshold`.

In [ ]:
for alignment in all_alignments:
    if alignment.cigarstring is None:
        continue
    position = alignment.reference_start
    cigar = re.findall("([0-9]+)([MIDNSHP])", alignment.cigarstring)
    sequence = alignment.query_sequence
    qual_string = alignment.qual
    offset = 0
    for aligned_chars, alignment_type in cigar:
        if alignment_type == 'H' or alignment_type == 'P':
            continue
        if alignment_type == 'N' or alignment_type == 'D':
            position += int(aligned_chars)
        elif alignment_type == 'S' or alignment_type == 'I':
            offset += int(aligned_chars)
        elif alignment_type == 'M':
            for _ in range(int(aligned_chars)):
                quality = phred_to_int(qual_string[offset])
                if position in pileup:
                    if quality >= quality_threshold:
                        pileup[position].append((alignment.qname, sequence[offset], quality))
                else:
                    if quality >= quality_threshold:
                        pileup[position] = [(alignment.qname, sequence[offset], quality)]
                position += 1
                offset += 1

La più piccola posizione nella reference coperta dalle read è:

In [ ]:
min_position = min(pileup.keys())
min_position

La più grande posizione nella reference coperta dalle read è:

In [ ]:
max_position = max(pileup.keys())
max_position

È quindi infine possibile consultare le basi impilate su una certa posizione della reference cambiando il valore di `queried_position`

In [ ]:
queried_position = max_position

read_list = pileup[queried_position]
count = Counter([read[1] for read in read_list])
print(count)
print("Number of reads in the queried position: " + str(len(read_list)))
for read in read_list:
    print("Query name: " + read[0] + "\t", "\tBase: " + read[1], "\tQuality: " + str(read[2]))